Baseline - Tn Promedio de 24 meses

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress
import os

# Optional for larger text and better visuals
sns.set(style="whitegrid", context="talk")

In [11]:
# Load the sales data (tab-delimited)
sales = pd.read_csv("datasets/sell-in.txt", sep="\t", dtype={"periodo": str})

# Load the stocks data (tab-delimited)
stocks = pd.read_csv("datasets/tb_stocks.txt", sep="\t", dtype={"periodo": str})

# Load the product information data (tab-delimited)
product_info = pd.read_csv("datasets/tb_productos.txt", sep="\t")

#carga productos a predecir
product_predict = pd.read_csv("datasets/product_id_apredecir201912.txt", sep="\t",header=0)

In [12]:
# Asegurar que 'periodo' sea string
sales['periodo'] = sales['periodo'].astype(str)



In [13]:
# Paso 1: agrupar por producto y periodo, sumando tn por mes
tn_por_producto_y_periodo = (
    sales
    .groupby(['product_id', 'periodo'])['tn']
    .sum()
    .reset_index()
)


In [14]:
# Paso 2: filtrar solo los últimos 3 meses deseados
ultimos_24 = ['201801', '201802', '201803','201804', '201805', '201806','201807', '201808', '201809','201810', '201811', '201812','201901', '201902', '201903','201904', '201905', '201906','201907', '201908', '201909','201910', '201911', '201912']
tn_filtrado = tn_por_producto_y_periodo[tn_por_producto_y_periodo['periodo'].isin(ultimos_24)]

In [15]:
# Paso 3: calcular el promedio de esos tres meses por producto
promedio_ultimos24 = (
    tn_filtrado
    .groupby('product_id')['tn']
    .mean()
    .reset_index()
    .rename(columns={'tn': 'tn'})
)

In [16]:
# Unir con productos a predecir
submission = product_predict.merge(promedio_ultimos24, on='product_id', how='left')
submission['tn'] = submission['tn'].fillna(0)

In [17]:
submission.head()

,product_id,tn
0,20001,1480.307860
1,20002,1131.400683
2,20003,829.972571
3,20004,640.487663
4,20005,622.990285


In [18]:
submission.to_csv("data/pred_trivial_media24.csv", index=False, encoding="utf-8")